# O problema

O problema que vamos resolver é um problema de detecção de fraudes em transações feitas por titulares de cartão europeus em 2023. A tarefa envolve classificar cada transação como fraudulenta (1) ou não fraudulenta (0). Este é um problema de classificação binária. Com base nas medidas diagnósticas contidas no dataset disponível em https://www.kaggle.com/datasets/nelgiriyewithana/credit-card-fraud-detection-dataset-2023/data . O data set já possui alguns tratamentos de dados, o que facilita a implementação. 

# Pacotes
Importações necessárias para a execução do código.

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow import keras

## Carregar o Conjunto de Dados
Fiz o download em zip e coloquei na mesma pasta deste notebook.
link para acessar no kagglehub: https://www.kaggle.com/datasets/nelgiriyewithana/credit-card-fraud-detection-dataset-2023 

## Pré-Processamentos dos Dados de Treino e de Teste


In [13]:
# Lendo o documento que está na mesma pasta deste notebook.
data = pd.read_csv('creditcard_2023.csv')
data.head() # lendo as linhas do dataset

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


# Preparação dos dados:
Selecionando as características (features) e a variável alvo (label).

In [77]:
# Separa a classe de outros atributos, aqui estou excluindo a coluna ID para não ser considerada como uma caracteristica (features).
# .values vai pegar só os valores, não vou manipular como um dataframe.
X = data.drop(['id', 'Class'], axis=1).values
y = data['Class'].values

Documentação de train_test_split: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
A função divide os dados em partições de treino e teste, de acordo com a proporção especificada pelo parâmetro test_size.
O parâmetro random_state é usado para deixar os resultados reproduzíveis para fins de avaliação do exercício.

In [49]:
# Preparando dataset para treino e test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Padronizando os atributos usando a média e a variância dos dados usando a função fit_transform: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [39]:
# Normalizando as informações.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [41]:
# Verificando
print("train_set:\n", X_train[:2,:])
print("\ntest_set:\n", X_test[:2,:])

train_set:
 [[ 0.31213157  0.25826656 -0.3501122   0.80569928  0.19632333 -0.14090144
  -0.08198516 -0.24044357 -0.78950374 -0.24426054  0.58558788 -0.58337337
  -0.53707759 -0.708119   -0.01514214 -0.29600118 -0.36841951 -0.10059766
  -0.0988151   0.42324328 -0.09934702 -0.10546591 -0.26123559 -0.53759301
   1.19277878  0.63844683  0.50812951  0.86840675 -0.26875907]
 [-0.73986278  0.88554241 -1.10148173  1.18401463 -0.58633832 -0.62762912
  -0.62891763  0.53928018 -0.95038128 -0.89128134  1.08526518 -1.1970575
   1.12783799 -1.1162116  -0.23108183 -0.96916907 -0.84629879 -0.55137186
   0.81594453 -0.14696334  0.32140502  0.53137689  0.16866097 -0.30406185
  -0.6662185   0.20720013 -0.13266645 -0.95144456  0.04090277]]

test_set:
 [[ 0.4207772  -0.0735515  -0.5643241   0.1876114  -0.00637572  0.43405352
  -0.36316872  0.09427327  0.07942915 -0.66748122  0.58293784 -0.74810797
  -1.03061828 -0.88516867  0.54548889 -1.0727201  -0.66091582 -1.14381418
  -0.17078924  0.23136694  0.1386495

Usando a função shape para obter o número de atributos e o número de exemplos de treinamento : https://numpy.org/devdocs/reference/generated/numpy.shape.html

In [51]:
# numero de atributos 
n = X_test.shape[1] # number of attributes
m = X_train.shape[0] # number of training examples

print ("Number of attributes: n = " + str(n))
print ("Number of training examples: m = " + str(m))
print ("Train set X shape: " + str(X_train.shape))
print ("Train set Y shape: " + str(y_train.shape))
print ("Test set X shape: " + str(X_test.shape))
print ("Test set Y shape: " + str(y_test.shape))

Number of attributes: n = 29
Number of training examples: m = 454904
Train set X shape: (454904, 29)
Train set Y shape: (454904,)
Test set X shape: (113726, 29)
Test set Y shape: (113726,)


## Definição do modelo
Crie um modelo em Keras com a seguinte configuração:
- Camada de entrada: no formato dos dados de entrada do problema
- Camada 1: 16 neurônios, função de ativação *relu*
- Camada 2: 32 neurônios, função de ativação *relu*
- Camada 3: 16 neurônios, função de ativação *relu*
- Camada 4 (saída): 1 neurônio, função de ativação *Sigmoid*

Referência de *Model*: https://keras.io/api/models/model/ <br>
Referência de funções de ativação disponíveis: https://keras.io/api/layers/activations/

In [63]:
inputs = keras.Input(shape=(X_train.shape[1],))
x = keras.layers.Dense(units=16, activation="relu")(inputs)  # Camada 1 com 3 neurônios
x = keras.layers.Dense(units=32, activation="relu")(x)       # Camada 2 com 5 neurônios
x = keras.layers.Dense(units=16, activation="relu")(x)       # Camada 3 com 3 neurônios
outputs = keras.layers.Dense(1, activation="sigmoid")(x)    # Camada de saída com 1 neurônio
model = keras.Model(inputs=inputs, outputs=outputs)         #camada de modelo

In [65]:
# Verificando
processed_data = model(X_train)
print(processed_data.shape)

(454904, 1)


In [67]:
# Imprime um resumo da rede, mostrando sua arquitetura e parâmetros.
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 29)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 16)                  │             480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 32)                  │             544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,569 (6.13 KB)

 Trainable params: 1,569 (6.13 KB)

 Non-trainable params: 0 (0.00 B)

# Compilação do modelo
Usando os seguintes parâmetros:
- Função de perda: binary_crossentropy
- Otimizador: Adam
- Métricas: accuracy, Precision, Recall

Referência da função *compile*: https://keras.io/api/models/model_training_apis/ <br>
Referência de funções de perda: https://www.tensorflow.org/api_docs/python/tf/keras/losses <br>
Referência de otimizadores: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers <br>
Referência de métricas: https://keras.io/api/metrics/

In [69]:
model.compile(optimizer='adam',loss='binary_crossentropy',  metrics=['accuracy', 'Precision', 'Recall'])

## Treinamento do modelo
Ajusta o modelo aos dados de treinamento. São fornecidos os dados de treinamento, o número de épocas (iterações) e o tamanho do lote (batch). Uma época é composta por uma única passagem por todos os exemplos do conjunto de treino. O tamanho do lote define o número de amostras (exemplos) a serem consideradas pelo modelo antes de atualizar os pesos. Assim, uma época é composta por um ou mais lotes.
O T treinamento do modelo udo os seguintes parâmetros:

Tamanho do lo32: 64
Número de época Usando use a função fit: https://keras.io/api/models/model_training_apis/

In [71]:
history = model.fit(X_train, y_train, batch_size=32, epochs=10)
print(history.history)  # print per-epoch timeseries of metrics values

Epoch 1/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - Precision: 0.8313 - Recall: 0.7925 - accuracy: 0.8158 - loss: 0.8408
Epoch 2/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - Precision: 0.9491 - Recall: 0.9190 - accuracy: 0.9348 - loss: 0.2057
Epoch 3/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - Precision: 0.9694 - Recall: 0.9270 - accuracy: 0.9489 - loss: 0.1397
Epoch 4/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - Precision: 0.9753 - Recall: 0.9322 - accuracy: 0.9544 - loss: 0.1235
Epoch 5/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - Precision: 0.9761 - Recall: 0.9337 - accuracy: 0.9555 - loss: 0.1194
Epoch 6/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - Precision: 0.9757 - Recall: 0.9357 - accuracy: 0.9563 - loss: 0.1170
Epoch 7/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - Precision: 0.9755 - Recall: 0.9374 - accuracy: 0.9570 - loss: 0.1135
Epoch 8/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - Precision: 0.9763 - Recall: 0.9381 - accuracy

## Avaliação do modelo
Avaliando o desempenho da rede no conjunto de teste com ase a função evaluate: https://keras.io/api/models/model_training_apis/

In [73]:
loss, acc, prec, rec = model.evaluate(X_test, y_test)
print("Loss: %.2f" % loss, "\nAccuracy: %.2f" % acc, "\nPrecision: %.2f" % prec, "\nRecall: %.2f" % rec)

3554/3554 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - Precision: 0.9809 - Recall: 0.9412 - accuracy: 0.9614 - loss: 0.1015
Loss: 0.10 
Accuracy: 0.96 
Precision: 0.98 
Recall: 0.94


## Predição
Apresenta a predição do conjunto de teste usandose a função predict: https://keras.io/api/models/model_training_apis/

In [75]:
predictions = model.predict(X_test)

print("Predictions: ", [round(x[0]) for x in predictions])
print("\nCorrect:     ", [round(x) for x in test_set_Y])

3554/3554 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
Predictions:  [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 

NameError: name 'test_set_Y' is not defined